In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

riotapi.set_region("OCE")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(False)

In [2]:
client = MongoClient()
loladb = client.loladb77
summoners_collection = loladb.summoners
matches_collection = loladb.matches

In [3]:
first_player = {}
first_player["name"] = "lolago"
first_player["id"] = ""
first_player["checked"] = False

if summoners_collection.find_one({'name':first_player["name"]}) == None:
    summoners_collection.insert_one(first_player)

In [4]:
#search for players based on game history
PLAYER_URL = "https://oce.op.gg/summoner/userName=%s"
DETAIL_URL = "https://oce.op.gg/summoner/matches/ajax/detail/gameId=%s&summonerId=%s"

current_summoner = None
for a_idx, a_summoner in enumerate(summoners_collection.find()):
    if a_summoner["checked"] == False:
        current_summoner = a_summoner
        break
        
starting_url = PLAYER_URL % current_summoner["name"]
print starting_url
r = requests.get(starting_url)
soup = BeautifulSoup(r.text, 'html.parser')

https://oce.op.gg/summoner/userName=Having Fun


In [5]:
def GetRowInfo(a_row):
    champion_name = a_row.find("td", class_="ChampionImage").find("a").find("div").text
    #print champion_name
    player_name = a_row.find("td", class_="SummonerName").find("a").text
    #print player_name
    player_tier = a_row.find("td", class_="Tier").text.strip()
    #print player_tier
    return (champion_name, player_name, player_tier)

def AddTeam(a_team):
    for a_member in a_team:
        a_summoner = {}
        a_summoner["name"] = a_member["player_name"]
        a_summoner["id"] = ""
        a_summoner["checked"] = False
        if summoners_collection.find_one({'name':a_summoner["name"]}) == None:
            summoners_collection.insert_one(a_summoner)

In [27]:
def CheckCurrentPlayer(current_summoner):
    starting_url = PLAYER_URL % current_summoner["name"]
    print starting_url
    r = requests.get(starting_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    match_list_block = soup.find("div", class_="GameItemList")
    if match_list_block == None:
        summoners_collection.delete_one({'name':current_summoner["name"]})
        return
    
    blocks = match_list_block.find_all("div", class_="GameItemWrap")
    for a_block_idx, a_block in enumerate(blocks):
        game_item = a_block.find("div", class_="GameItem")

        game_result_part = a_block.find("div", class_="GameStats")
        game_result = game_result_part.find("div", class_="GameResult")
        game_result_text = game_result.text.strip()
        if game_result_text == "Remake":
            continue

        game_type_text = game_result_part.find("div", class_="GameType").text.strip()
        if game_type_text != "Ranked Solo":
            continue

        stats_button = game_item.find("div", class_="StatsButton")
        stats_link = stats_button.find_all("a")[-1]
        stats_content = stats_link.attrs['onclick']
        stats_list = stats_content.split("(")[1].split(")")[0].split(",")

        game_id = stats_list[0].lstrip()
        summoner_id = stats_list[1].lstrip()

        current_summoner["id"] = summoner_id

        match_record = {}
        match_record["match_id"]= game_id

        match_url = DETAIL_URL % (game_id, summoner_id)
        #print a_block_idx, match_url

        detail_r = requests.get(match_url)
        detail_soup = BeautifulSoup(detail_r.text, 'html.parser')

        detail_div = detail_soup.find("div", class_="GameDetailTableWrap")
        detail_tables = detail_div.find_all("table", class_="GameDetailTable")
        blue_win = detail_tables[0].attrs['class'][1] == 'Result-WIN'

        if blue_win:
            match_record["match_winner"] = 0
        else:
            match_record["match_winner"] = 1

        blue_team = []
        red_team = []
        blue_rows = detail_tables[0].find_all("tr", class_="Row")
        #row 1~5 are players stats
        for a_row_index in range(1, 6):
            a_row = blue_rows[a_row_index]
            (champion_name, player_name, player_tier) = GetRowInfo(a_row)
            blue_team.append({"champion_name":champion_name, "player_name":player_name, "player_tier":player_tier})
        red_rows = detail_tables[1].find_all("tr", class_="Row")
        for a_row_index in range(1, 6):
            a_row = red_rows[a_row_index]
            (champion_name, player_name, player_tier) = GetRowInfo(a_row)
            red_team.append({"champion_name":champion_name, "player_name":player_name, "player_tier":player_tier})

        match_record["blue_team"] = blue_team
        match_record["red_team"] = red_team

        AddTeam(blue_team)
        AddTeam(red_team)

        #print match_record
        if matches_collection.find_one({'match_id':match_record["match_id"]}) == None:
            matches_collection.insert_one(match_record)

    summoners_collection.update_one({'name':current_summoner["name"]}, {'$set': {'id': current_summoner["id"]}})
    summoners_collection.update_one({'name':current_summoner["name"]}, {'$set': {'checked': True}})

In [32]:
while True:
    current_summoner = None
    for a_idx, a_summoner in enumerate(summoners_collection.find()):
        if a_summoner["checked"] == False:
            current_summoner = a_summoner
            break
    if current_summoner == None:
        break
    print "Checking " + current_summoner["name"]
    CheckCurrentPlayer(current_summoner)

Checking xGVCx Scout
https://oce.op.gg/summoner/userName=xGVCx Scout
Checking Cancel
https://oce.op.gg/summoner/userName=Cancel
Checking Fizz In My Jayce
https://oce.op.gg/summoner/userName=Fizz In My Jayce
Checking Gud Lux Have Pun
https://oce.op.gg/summoner/userName=Gud Lux Have Pun
Checking CBXtreme
https://oce.op.gg/summoner/userName=CBXtreme
Checking Kangaroo Killer
https://oce.op.gg/summoner/userName=Kangaroo Killer
Checking MagicalMarshmall
https://oce.op.gg/summoner/userName=MagicalMarshmall
Checking Bow to Leona
https://oce.op.gg/summoner/userName=Bow to Leona
Checking mens in yellow
https://oce.op.gg/summoner/userName=mens in yellow
Checking ADDSY
https://oce.op.gg/summoner/userName=ADDSY
Checking Elea
https://oce.op.gg/summoner/userName=Elea
Checking TracyLY
https://oce.op.gg/summoner/userName=TracyLY
Checking MadTuNEs
https://oce.op.gg/summoner/userName=MadTuNEs
Checking Armitage
https://oce.op.gg/summoner/userName=Armitage


KeyboardInterrupt: 